In [1]:
import os
import cv2
import sys
import torch
from model import Encoder, Decoder
import numpy as np
from torch import nn
from os import listdir
from seq2seq_parts import *
from os.path import isfile, isdir

import matplotlib.pyplot as plt
from convlstm2d import ConvLSTMCell
# from data_preprosess import get_clean_data
from torch.optim import Adam, RMSprop, SGD

In [2]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, 'data/train')

images_paths = [os.path.join(data_dir, f) for f in listdir(data_dir) if isfile(os.path.join(data_dir, f))]

In [3]:
IMAGES_WIDTH = 256
IMAGES_HEIGHT = 256
IMAGES_DEPTH = 1

TIME_STEMP = 6
np.random.seed(14)

In [4]:
features_list = np.zeros((len(images_paths), IMAGES_WIDTH, IMAGES_HEIGHT, IMAGES_DEPTH), dtype = np.uint8)

def get_feature_images(paths, features_list):
    for i, path in enumerate(paths):
        image = cv2.imread(path, -1)
        image = cv2.resize(image, (IMAGES_WIDTH, IMAGES_HEIGHT))
        image = image.reshape(IMAGES_WIDTH, IMAGES_HEIGHT, IMAGES_DEPTH)
        image = image.astype(np.uint8)
        features_list[i] = image
    return features_list

    
sys.stdout.flush()
    
print('Getting Training Data...')

data = get_feature_images(images_paths, features_list)

print('Training data loading completed!')


print('we have {} Training Images Found.'.format(len(data)))

print('All data completely loaded!')

Getting Training Data...
Training data loading completed!
we have 14566 Training Images Found.
All data completely loaded!


In [5]:
X_train = data / 255
print('Data Preprosessing complete!')

Data Preprosessing complete!


In [6]:
def get_proper_preprosess_data(data, frame, X_train):
    length = len(data)
    imgs = []
    lbls = []
    cnt_y = 0
    for i in range(0, length, frame):
        images = []
        labels = []
        cnt = 0
        
        for j in range(i, i + frame):
            
            if j < length:
                images.append(X_train[j])
            else:
                images.append(X_train[cnt])
                cnt += 1   
            
            le = j+1
            if le < length:
                try:
                    labels.append(data[le])
                except:
                    labels.append(data[0])
            else:
                labels.append(data[cnt_y])
                cnt_y +=1
                
        imgs.append(images)
        lbls.append(labels)
        
    return imgs, lbls
  
train_X, train_Y = get_proper_preprosess_data(data, TIME_STEMP, X_train)

In [7]:
train_X = np.array(train_X)
train_Y = np.array(train_Y)

In [8]:
X_train = train_X[:int(len(train_X) * 0.8)]
Y_train = train_Y[:int(len(train_X) * 0.8)]

val_X = train_X[int(len(train_X) * 0.91):]
val_Y = train_Y[int(len(train_X) * 0.91):]

test_X = train_X[int(len(train_X) * 0.9):]
test_Y = train_Y[int(len(train_X) * 0.9):]

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    output_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [torch.from_numpy(features[start_i:end_i]).type(torch.FloatTensor), torch.from_numpy(labels[start_i:end_i]).type(torch.FloatTensor)]
        output_batches.append(batch)
        
    return output_batches

batch_size = 8
train_loader = batches(batch_size, X_train, Y_train)
val_loader = batches(batch_size, val_X, val_Y)

In [11]:
#Drop last batch
last_train_batch = train_loader.pop(-1)
last_validation_batch= val_loader.pop(-1)

In [12]:
import random
random.shuffle(train_loader)
random.shuffle(val_loader)

In [13]:
import random


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
      
    #initialize the encoder and decoder
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, features, targets, teacher_forcing_ratio=0.5):
        
        batch_size = targets.shape[1]
        time = targets.shape[0]
        
        outputs = torch.FloatTensor(torch.zeros((time, batch_size, IMAGES_DEPTH, IMAGES_WIDTH, IMAGES_HEIGHT))).to(device)
        
        for t in range(time):
            x1, x2, x3, x4, x5, output, states = self.encoder(features[t])
            
        decoder_hidden = states
        layers = [x1, x2, x3, x4, x5]
        output_encoder = output
        
        input = torch.FloatTensor(torch.zeros(output.shape)).to(device)
        
        for t in range(1, time):
            decoder_output, decoder_hidden = self.decoder(layers[0], layers[1], layers[2], layers[3], layers[4], input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            input = (output_encoder if teacher_force else decoder_hidden[0])
            
        return outputs

In [14]:
from torch.optim.lr_scheduler import StepLR
def get_loss_per_epoch(model, feature, target, model_optimizer, criterion):
    model_optimizer.zero_grad()

    time_stemp = feature.size(0)
    loss = 0
    epoch_loss = 0

    output = model(feature, target)

    target_time_stemp = output.size(0)

    #calculate the loss from a predicted sentence with the expected result
    for t in range(target_time_stemp):
        loss += criterion(output[t], target[t])

    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / target_time_stemp

    return epoch_loss

In [15]:
def trainModel(model, inputs, targets, num_iteration=10000):
    model.train()

    optimizer = Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    total_loss_iterations = 0

#     training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
#                      for i in range(num_iteration)]

    for iter in range(1, num_iteration+1):
        input_pair = inputs[iter - 1]
        targets_pair = targets[iter - 1]
        input_tensor = input_pair.view(6, 1, 1, 256, 256).to(device)
        target_tensor = targets_pair.view(6, 1, 1, 256, 256).to(device)

        loss = get_loss_per_epoch(model, input_tensor, target_tensor, optimizer, criterion)

        total_loss_iterations += loss

        if iter % 500 == 0:
            avarage_loss= total_loss_iterations / 5000
            total_loss_iterations = 0
            print('%d %.4f' % (iter, avarage_loss))

    torch.save(model.state_dict(), 'mytraining.pt')
    return model

In [16]:
encoder = Encoder()
encoder = encoder.to(device)

decoder = Decoder()
decoder = decoder.to(device)

seq2seq = Seq2Seq(encoder, decoder, device).to(device)

In [17]:
seq2seq.train()
criterion = nn.MSELoss()
learning_rate = 1e-04

optimizer = torch.optim.SGD(seq2seq.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# optimizer = Adam(seq2seq.parameters(), lr=1e-3)

In [18]:
epochs = 51

In [ ]:
for epoch in range(epochs):
        
    total_loss_iterations = 0
    for x, y in train_loader:
        for i in range(len(x)):
            tensor = x[i].view(6, 1, 1, 256, 256).cuda()
            label = y[i].view(6, 1, 1, 256, 256).cuda()

            loss = get_loss_per_epoch(seq2seq, tensor, label, optimizer, criterion)
            total_loss_iterations += loss
    else:
        print('Epochs : 51/{}'.format(epoch), '...Training loss : {0:.4f}'.format(total_loss_iterations/14566))
    torch.save(seq2seq.state_dict(), 'seq2seq_by_jimit_jayswal.pt')

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Epochs : 51/0 ...Training loss : 1455.4497
Epochs : 51/1 ...Training loss : 1427.7880
Epochs : 51/2 ...Training loss : 1427.6111
Epochs : 51/3 ...Training loss : 1427.5617
Epochs : 51/4 ...Training loss : 1427.4975
Epochs : 51/5 ...Training loss : 1427.3347
Epochs : 51/6 ...Training loss : 1427.1286
Epochs : 51/7 ...Training loss : 1427.0143
Epochs : 51/8 ...Training loss : 1426.8967
Epochs : 51/9 ...Training loss : 1426.7987
Epochs : 51/10 ...Training loss : 1426.7333
Epochs : 51/11 ...Training loss : 1426.6647
Epochs : 51/12 ...Training loss : 1426.5900
Epochs : 51/13 ...Training loss : 1426.5119
Epochs : 51/14 ...Training loss : 1426.4393
Epochs : 51/15 ...Training loss : 1426.3755
Epochs : 51/16 ...Training loss : 1426.3164
